# Contextualized Prompts 
This tutorial guides you through creating an LLM generated restaurant recommendation function.
This is an example of how Tecton managed and contextualized prompts enable personalization.

It uses Tecton's real-time enriched prompts to provide current context to the LLM in order to improve the quality of its response. 
This tutorial demonstrates both LangChain and LlamaIndex integration with Tecton prompts.

## Install Packages

In [ ]:
!pip install 'tecton[rift]==0.10.0b32' gcsfs s3fs --force-reinstall
!pip install "$HOME/Downloads/tecton_utils-0.0.4-py3-none-any.whl" --force-reinstall

!pip install openai -q
!pip install langchain-openai -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain_core -q

# Tecton Prompt

In the following cell you'll create a Tecton Agent with a system prompt that provides instructions to the LLM. The instructions are parameterized with a specific user's data. 

The agent creation function takes a Tecton feature view as input which is used at run-time to acquire the latest values of the parameters for the user.

In [ ]:
from tecton_utils.ai import AgentClient, AgentService, prompt
from tecton_utils.core_utils import make_request_source



def restaurant_recommender_agent( user_info):    
    
    location_request = make_request_source(location = str)

    @prompt(sources=[ location_request, user_info])
    def sys_prompt(location_request, user_info ):
        name = user_info["name"]
        food_preference = user_info["food_preference"]
        location = location_request["location"]
        return f"""
        You are a consierge service that recommends restaurants.
        You are serving {name}. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in {location}. 
        If the user does not provide a cuisine or food preference, choose a {food_preference} restaurant.
        """
        
    return AgentService(
        name="restaurant_recommender",
        prompts=[ sys_prompt],
        tools=[user_info],
    )

The example above uses a single feature view as input. Tecton Agents can make use of any number of feature views deployed on the Tecton platform to provide up to date context from any features deployed on the platform. 

Notice that the `sys_prompt` function additionally takes the `location` parameter in the prompt. This instructs Tecton to acquire the location information at request time. Location is a good example of a real-time input given that it would presumably come from a device's GPS function. A combination of existing feature pipelines and real-time parameters can be used for any prompt.

## Sample Data

In order to keep this notebook self-contained, you will create a mock feature view with some hard-coded data.
In a real application, you would use Feature Views that continuously update feature values and therefore provide up-to-date context to the LLM application.

In [ ]:
import pandas as pd
from tecton import RequestSource
from tecton.types import Field, String


from tecton_utils.testing import mock_batch_feature_view


mock_data = pd.DataFrame(
        [
            {
                "user_id": "user1",
                "name": "Jim",
                "age": 30,
                "food_preference": "American",
            },
            {
                "user_id": "user2",
                "name": "John",
                "age": 40,
                "food_preference": "Italian",
            },
            {
                "user_id": "user3",
                "name": "Jane",
                "age": 50,
                "food_preference": "Chinese",
            },
        ]
    )

user_preference_fv = mock_batch_feature_view(
        "user_info", mock_data, entity_keys=["user_id"], description="User's profile with name, age and food preference."
    )

The feature view identifies the key `user_id` that is needed to access a user's data, this attribute must be provided when using the feature view in a prompt. 

In the following cell, you will test the prompt through an AgentClient's invoke_prompt method using a `user_id` and a `location` value. The `user_id` is used to retrieve a specific user's values. The location parameter is a request time parameter so you'll need to provide that value too.

In [ ]:
# create the Tecton Agent
recommender_agent = restaurant_recommender_agent(user_preference_fv )

# create a client to invoke with the agent
client = AgentClient.from_local( recommender_agent )

#test the agent using "sys_prompt" prompt
print(client.invoke_prompt("sys_prompt", user_id="user3", location="Chicago"))


## Incorporate Contextualized Prompt into a LangChain agent

The Tecton AgentClient can be used to create a LangChain agent which will use the enriched prompt to generate a response.
In the cell below you will instantiate an LLM model using OpenAI.

Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.

In [ ]:
import openai as oa
import os
from langchain_openai import ChatOpenAI


# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# instantiate LLM model
gpt_llm = ChatOpenAI(model="gpt-4o-mini")

#create a lang chain agent that uses the system_prompt 
lc_agent = client.make_langchain_agent(llm=gpt_llm, system_prompt = "sys_prompt")

## Test it out

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

In [ ]:
response = lc_agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user1", "location":"Charlotte, NC"} 
)
print(response)

In [ ]:
response = lc_agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user1", "location":"New York, NY"} 
)
print(response)

In [ ]:
response = lc_agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user2", "location":"Charlotte, NC"} 
)
print(response)

In [ ]:
response = lc_agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user3", "location":"Charlotte, NC"} 
)
print(response)

# Conclusion

Tecton prompts are used to incorporate real-time, streaming and batch features into your generative AI applications, providing a great solution for personalization. In general, it can be used to provide up to date context for any LLM driven function. 
